In [ ]:
#!pip install pyarrow
#!pip install chromadb
#!pip install sentence_transformers
#!pip install tfidf_index
#!pip install pandas
#!pip install numpy
#!pip install datasets

  Using cached fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
Using cached fsspec-2025.9.0-py3-none-any.whl (199 kB)
Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
Using cached aiosignal-1.4.0-py3-none-any.whl (7.5 kB)

  Attempting uninstall: fsspec

    Found existing installation: fsspec 2025.10.0

    Uninstalling fsspec-2025.10.0:

      Successfully uninstalled fsspec-2025.10.0

   ---------- -----------------------------  3/12 [fsspec]
   ---------- -----------------------------  3/12 [fsspec]
   ------------- --------------------------  4/12 [frozenlist]
   ---------------- -----------------------  5/12 [dill]
   ----------------------- ----------------  7/12 [yarl]
   -------------------------- -------------  8/12 [multiprocess]
   --------------------------------- ------ 10/12 [aiohttp]
   ---------------------------------

# Step - 1 Load the data

1a - Filtered for task-1, and 'us' and 'e' esci (exact, substitute, complement, and irrelevant)

1b - Merge the Datasets using product_locale and product_id (connects each query/label pair to its corresponding product information)

1c - Filtering the large merged DataFrame down to the required data




In [1]:
import pandas as pd
import numpy as np
import os

df_examples =pd.read_parquet(r'C:\Users\saiasg\OneDrive - kochind.com\Desktop\Projects\esci\esci_dataset\dataset\shopping_queries_dataset_examples.parquet')
df_products = pd.read_parquet(r'C:\Users\saiasg\OneDrive - kochind.com\Desktop\Projects\esci\esci_dataset\dataset\shopping_queries_dataset_products.parquet')

In [2]:
df_merged = pd.merge(
    df_examples,
    df_products,
    how='left',
    left_on=['product_locale','product_id'],
    right_on=['product_locale', 'product_id']
)

# filter for Task 1 (using the 'small_version' flag)
df_task_1 = df_merged[df_merged["small_version"] == 1]

# filter for the training set
df_task_1_train = df_task_1[df_task_1["split"] == "train"]

# applying filters for 'us' locale and 'E' label
final_training_data = df_task_1_train[
    (df_task_1_train["product_locale"] == "us") &
    (df_task_1_train["esci_label"] == "E")
]

In [3]:
# final_training_data
final_training_data.head()

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
17,17,!awnmower tires without rims,1,B08L3B9B9P,us,E,1,1,train,MaxAuto 2-Pack 13x5.00-6 2PLY Turf Mower Tract...,MaxAuto 2-Pack 13x5.00-6 2PLY Turf Mower Tract...,Please check your existing tire Sidewall for t...,MaxAuto,None
20,20,!awnmower tires without rims,1,B07C1WZG12,us,E,1,1,train,(Set of 2) 15x6.00-6 Husqvarna/Poulan Tire Whe...,No fuss. Just take off your old assembly and r...,Tire size:15x6.00-6 Ply: 4 Tubeless\n6x4.5 Whe...,Antego Tire & Wheel,Husqvarna Silver
21,21,!awnmower tires without rims,1,B077QMNXTS,us,E,1,1,train,MaxAuto 2 Pcs 16x6.50-8 Lawn Mower Tire for Ga...,<br>Tire Specifications:<br> 1. Material: Rubb...,"Set of 2 16X6.50-8, 16x6.50x8, 16-6.50-8 Lawn ...",MaxAuto,Black
23,23,!awnmower tires without rims,1,B06XX6BM2R,us,E,1,1,train,"MARASTAR 21446-2PK 15x6.00-6"" Front Tire Assem...",None,Tire: 2 pack 15x6. 00-6 tube-type turf SAVER t...,MARASTAR,None
26,26,!awnmower tires without rims,1,B0089RNSNM,us,E,1,1,train,Honda 42710-VE2-M02ZE (Replaces 42710-VE2-M01Z...,Honda 42710-VE2-M02ZE (Replaces 42710-VE2-M01Z...,Set of 2 Honda OEM Rear Wheels\nReplaces 42710...,Honda,None


# Step - 2: Create sample dataset

Creating a 500-row sample from the training data, centered on 50 unique queries.

2a: Random sampling of rows

2b: (Simple Random Sampling), select 50 unique queries and then sample row

2c: Stratified sampling with around 60 unique queries to find 50 that yield a sample close to 500 rows

In [ ]:
df_full = final_training_data.reset_index(drop=True)
print(f"1. Total rows in dataset: {len(df_full)}")

1. Total available rows in master dataset: 181819


#### 2a: Random sampling of rows - Basic barebones

We just randomly grab 500 rows

In [ ]:
# --- Configuration ---
RANDOM_STATE = 42
TARGET_QUERIES = 50
TARGET_ROWS = 500

df_full = final_training_data.reset_index(drop=True)

print(f"\n--- Step 2a: Simple Random Sample ---")
sample_2a = df_full.sample(n=TARGET_ROWS, random_state=RANDOM_STATE).reset_index(drop=True)
final_unique_2a = sample_2a['query'].nunique()

print(f"Result: {final_unique_2a} unique queries in the sample.")
print("\nSample Head:")
display(sample_2a[['query', 'product_title']].head())

1. Total available rows in master dataset: 181819

--- Step 2a: Simple Random Sample ---
We took a simple random sample of 500 rows from the full dataset.
Result: 490 unique queries in the sample.

Sample 2a Head:


,query,product_title
0,drainer for sink,Outivity Sink Drain Shelf Triangular Sink Bask...
1,chernobyl,Azure Dust: Inside Chernobyl's Exclusion Zone
2,retaine eye drops for dry eyes preservative free,UrsaPharm Hylo-Forte Lubricating Eye Drops 10M...
3,long sleeve pink dress,R.Vivimos Women's Autumn Winter Cotton Long Sl...
4,(hearing aid not amplifer),"Hearing Aids, Enjoyee Hearing Aids for Seniors..."


#### Step 2b: Simple Random Sample



In [ ]:
print(f"\n--- Step 2b: Sample from Filtered Queries ---")

# 1. randomly select 50 unique queries
unique_queries_2b = df_full['query'].unique()
if len(unique_queries_2b) < TARGET_QUERIES:
    sample_queries_2b = pd.Series(unique_queries_2b)
else:
    sample_queries_2b = pd.Series(unique_queries_2b).sample(n=TARGET_QUERIES, random_state=RANDOM_STATE)

# 2. filter master set to products matching these queries
df_filtered_2b = df_full[df_full['query'].isin(sample_queries_2b)].reset_index(drop=True)
total_rows_for_queries_2b = len(df_filtered_2b)

print(f"-> selected {len(sample_queries_2b)} unique queries.")
print(f"-> total rows available after filtering: {total_rows_for_queries_2b}")

# 3. sample 500 rows from filtered dataset
if total_rows_for_queries_2b <= TARGET_ROWS:
    sample_2b = df_filtered_2b.copy()
    print(f"filtered set too small ({total_rows_for_queries_2b} rows), using all available.")
else:
    sample_2b = df_filtered_2b.sample(n=TARGET_ROWS, random_state=RANDOM_STATE).reset_index(drop=True)
    print(f"took random sample of {len(sample_2b)} rows from filtered dataset.")

# 4. final results
final_unique_2b = sample_2b['query'].nunique()
print(f"result: {len(sample_2b)} rows and {final_unique_2b} unique queries in sample.")

print("\nSample 2b Head:")
display(sample_2b[['query', 'product_title']].head())


--- Step 2b: Sample from Filtered Queries ---
-> We selected 50 unique queries.
-> Total associated rows available after filtering: 329
Filtered set is too small (329 rows), using all available rows.
Result: 329 rows and 50 unique queries in the sample.

Sample 2b Head:


,query,product_title
0,6 dining chairs,Yaheetech Dining Chairs Velvet Armchairs for C...
1,6 dining chairs,CozyCasa Dining Chairs Modern Style Dining Cha...
2,6 dining chairs,Yaheetech Dining Chairs with Waterproof leathe...
3,6 dining chairs,Yaheetech Dining Chairs Dining Room Chairs Liv...
4,6 dining chairs,Modern Dining Chairs Set of 6 - Faux Leather D...


#### 2c Stratified sampling

Using this approach to select queries by row count to maximize sample diversity

To ensures representation from all queries, and also high-frequency queries, using this approach

In [ ]:
print(f"\n--- Step 2c: Stratified Sampling for Target Rows and Queries ---")

TARGET_QUERIES_2c = 70 # aiming for higher number of unique queries initially

# 1. randomly selecting 70 unique queries
unique_queries_full = df_full['query'].unique()
if len(unique_queries_full) < TARGET_QUERIES_2c:
    potential_sample_queries = pd.Series(unique_queries_full)
    print(f"only {len(unique_queries_full)} unique queries available, using all.")
else:
    potential_sample_queries = pd.Series(unique_queries_full).sample(n=TARGET_QUERIES_2c, random_state=RANDOM_STATE)
    print(f"randomly selected {len(potential_sample_queries)} potential unique queries.")

# 2. filtering rows associated with selected queries
df_potential_queries = df_full[df_full['query'].isin(potential_sample_queries)].reset_index(drop=True)
print(f"filtered down to {len(df_potential_queries)} rows associated with these queries.")

# 3. calculate rows per query
query_row_counts = df_potential_queries.groupby('query').size().sort_values(ascending=False)
print("\nRow counts for potential queries:")
display(query_row_counts.head())

# 4. select top 50 queries by row count
selected_queries_2c = []
current_row_count = 0
for query, count in query_row_counts.items():
    if len(selected_queries_2c) < 50:
         selected_queries_2c.append(query)
         current_row_count += count
    else:
        pass

# ensure we have exactly 50 queries if available
if len(selected_queries_2c) < 50 and len(potential_sample_queries) >= 50:
     print(f"warning: could only select {len(selected_queries_2c)} queries.")
elif len(selected_queries_2c) == 50:
     print(f"\nselected 50 queries that yield {current_row_count} rows.")

# 5. filter to final 50 queries
df_final_50_queries = df_potential_queries[df_potential_queries['query'].isin(selected_queries_2c)].reset_index(drop=True)

# 6. determine final sample based on row count
if len(df_final_50_queries) == TARGET_ROWS:
    sample_2c = df_final_50_queries.copy()
    print(f"final filtered set has exactly {TARGET_ROWS} rows.")
elif len(df_final_50_queries) < TARGET_ROWS:
    sample_2c = df_final_50_queries.copy()
    print(f"final filtered set has {len(df_final_50_queries)} rows, using all available.")
else:
    sample_2c = df_final_50_queries.sample(n=TARGET_ROWS, random_state=RANDOM_STATE).reset_index(drop=True)
    print(f"final filtered set has {len(df_final_50_queries)} rows, taking random sample of {TARGET_ROWS}.")

# 7. final results
final_unique_2c = sample_2c['query'].nunique()
print(f"\nfinal sample (2c) result: {len(sample_2c)} rows and {final_unique_2c} unique queries.")

print("\nFinal Sample (2c) Head:")
display(sample_2c[['query', 'product_title', 'esci_label']].head())


--- Step 2c: Stratified Sampling for Target Rows and Queries ---
Randomly selected 70 potential unique queries.
Filtered down to 520 rows associated with these potential queries.

Row counts for potential queries:


query
turning shoe                         30
6 dining chairs                      22
plants                               22
tan and brown bathroom wall decor    21
poleras deportivas mujer             19
dtype: int64


Selected 50 queries that yield a total of 485 rows.
Final filtered set has 485 rows, less than the target 500. Using all available rows.

Final Sample (2c) Result: 485 rows and 50 unique queries.

Final Sample (2c) Head:


,query,product_title,esci_label
0,6 dining chairs,Yaheetech Dining Chairs Velvet Armchairs for C...,E
1,6 dining chairs,CozyCasa Dining Chairs Modern Style Dining Cha...,E
2,6 dining chairs,Yaheetech Dining Chairs with Waterproof leathe...,E
3,6 dining chairs,Yaheetech Dining Chairs Dining Room Chairs Liv...,E
4,6 dining chairs,Modern Dining Chairs Set of 6 - Faux Leather D...,E


In [ ]:
# Save the sample_2c DataFrame to a CSV file
sample_2c.to_csv(r'C:\Users\saiasg\OneDrive - kochind.com\Desktop\Projects\esci\esci_dataset\sample_2c_full_data.csv', index=False)

sample_2c.head()

### Some Data Explroing

Intresting queires: One query was  "usb2aub2ra1m"

And its apprently a product id for right anlged usb connector

https://www.startech.com/en-eu/cables/usb2aub2ra1m?srsltid=AfmBOoryvB93OxhVQnPUAocknMNz41MVDvr2TJMrWf0ijRnCwf5htlXn

Face urine? - Fake urine but still never knew these existed haha

And some plumbing related queires: zurn qkipsp 5 port plastic manifold without valves

# Step 3: Vector Index

3a - Baseline tf-idf 

3b - Dense model (all-MiniLM-L6-v2)

3c - Two tower dense model

3d - Hybrid model


In [8]:
import pandas as pd
import numpy as np
import os
import warnings
import chromadb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import shutil
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sentence_transformers import CrossEncoder

c:\Users\saiasg\AppData\Local\miniconda3\envs\esci\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# loading the stored dataset
df_sample_2c = pd.read_csv(r'C:\Users\saiasg\OneDrive - kochind.com\Desktop\Projects\esci\esci_dataset\sample_2c_full_data.csv')

df_sample_2c.head()

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
0,118060,6 dining chairs,4845,B08CZ6TC2L,us,E,1,1,train,Yaheetech Dining Chairs Velvet Armchairs for C...,Set of 6 Kitchen Dining Chairs for Counter Lou...,STRONG METAL LEGS: To enhance the weight capac...,Yaheetech,Grey
1,118064,6 dining chairs,4845,B08HQG1MFS,us,E,1,1,train,CozyCasa Dining Chairs Modern Style Dining Cha...,<b>If you are in search of some quality-reliab...,Dining Chairs set of 6 -- White PP backrest an...,CozyCasa,White
2,118065,6 dining chairs,4845,B08K2K3J4C,us,E,1,1,train,Yaheetech Dining Chairs with Waterproof leathe...,Make every long-time sitting comfortable. The ...,MULTIPLE USE: Sold in a set of 6 chairs. Desig...,Yaheetech,Brown
3,118066,6 dining chairs,4845,B08K2V66N8,us,E,1,1,train,Yaheetech Dining Chairs Dining Room Chairs Liv...,Make every dinner time comfortable. Constructe...,MULTIPLE USE: Sold in a set of 6 chairs. This ...,Yaheetech,Khaki
4,118067,6 dining chairs,4845,B08K8VDTW8,us,E,1,1,train,Modern Dining Chairs Set of 6 - Faux Leather D...,<b>Modern Dining Chairs Set of 6 - Faux Leathe...,Comfortable Dining Chairs Set of 6 - The dinin...,WENYU,Grey


In [ ]:
# create product corpus (de-duplicated products)
print("Processing data into a unique Product Corpus...")
product_columns = [
    'product_id', 
    'product_title', 
    'product_description', 
    'product_bullet_point', 
    'product_brand', 
    'product_color'
]
product_corpus_df = df_sample_2c[product_columns].drop_duplicates(subset=['product_id']).reset_index(drop=True)

# fill nans with empty strings
text_cols_to_fill = product_columns[1:]
for col in text_cols_to_fill:
    product_corpus_df[col] = product_corpus_df[col].fillna('')

# combine all text fields into single product_text for embedding
product_corpus_df['product_text'] = (
    product_corpus_df['product_title'] + ' ' +
    product_corpus_df['product_brand'] + ' ' +
    product_corpus_df['product_color'] + ' ' +
    product_corpus_df['product_description'] + ' ' +
    product_corpus_df['product_bullet_point']
)
# cleaning up extra whitespaces
product_corpus_df['product_text'] = product_corpus_df['product_text'].str.replace(r'\s+', ' ', regex=True).str.strip()
print(f"created corpus of {len(product_corpus_df)} unique products.")

# creating query evaluation set (query-to-product pairs)
query_eval_set = df_sample_2c[['query', 'query_id', 'product_id', 'esci_label']].copy()
print(f"created evaluation set of {len(query_eval_set)} query-product pairs.")

Processing data into a unique Product Corpus...
Created a corpus of 485 unique products.
Created an evaluation set of 485 query-product pairs.


## 3a: tf-idf

In [11]:
documents = product_corpus_df['product_text'].tolist()

print(f"Creating TF-IDF embeddings (sparse vectors) for {len(documents)} documents...")

# stop words and limit to the top 5000 most frequent terms
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# Create the sparse TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
print(f"TF-IDF matrix created with shape: {tfidf_matrix.shape}")

# Using 'cosine' similarity and 'brute' force
print("Building in-memory sparse index with scikit-learn (NearestNeighbors)...")
n_neighbors = 10
nn_index = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine', algorithm='brute')
nn_index.fit(tfidf_matrix)
print("In-memory sparse index built successfully.")

Creating TF-IDF embeddings (sparse vectors) for 485 documents...
TF-IDF matrix created with shape: (485, 5000)
Building in-memory sparse index with scikit-learn (NearestNeighbors)...
In-memory sparse index built successfully.


In [ ]:
test_query = "carpenter bench press" # Not using a direct numeric or direct word serach

# Find the ground truth for this query from our in-memory DataFrame
ground_truth_df = query_eval_set[query_eval_set['query'] == test_query]
ground_truth_ids = ground_truth_df['product_id'].tolist()

print(f"Test Query: '{test_query}'")
print(f"Ground Truth 'Exact' Product IDs ({len(ground_truth_ids)}): {ground_truth_ids}")

# Embed the test query *using the same vectorizer*
# .transform returns a sparse 2D matrix
query_vector = tfidf_vectorizer.transform([test_query])

# Search the index
print(f"\nSearching index for Top {n_neighbors} results...")
# .kneighbors returns (distances, indices)
distances, indices = nn_index.kneighbors(query_vector)

# Flatten the results from 2D to 1D
result_indices = indices[0]
result_distances = distances[0]


Test Query: 'carpenter bench press'
Ground Truth 'Exact' Product IDs (8): ['B088R8VC7V', 'B07N4QN64D', 'B01M4F8JZJ', 'B01LZV1QW1', 'B00HQONFVE', 'B00BHSPJC8', 'B00068U7XQ', 'B00SIQ1DLS']

Searching index for Top 10 results...


In [ ]:
# Get the full product info from the original corpus_df using the indices
results_df = product_corpus_df.iloc[result_indices].copy()

# Add the distance and a ground_truth check
results_df['_distance'] = result_distances
results_df['is_ground_truth'] = results_df['product_id'].isin(ground_truth_ids)

print(results_df[['product_id', '_distance', 'is_ground_truth', 'product_title']])
matches_in_top_10 = results_df['is_ground_truth'].sum()
print(f"\nFound {matches_in_top_10} out of {len(ground_truth_ids)} ground truth items in the Top 10 results.")

Search Results (lower distance is better):
     product_id  _distance  is_ground_truth  \
73   B01LZV1QW1   0.813588             True   
70   B088R8VC7V   0.868601             True   
76   B00068U7XQ   0.883066             True   
431  B06XD1DVBG   0.885864            False   
71   B07N4QN64D   0.893176             True   
427  B06XD5G2XB   0.911316            False   
77   B00SIQ1DLS   0.912220             True   
74   B00HQONFVE   0.916837             True   
129  B07G15QZNQ   0.924956            False   
342  B0015TX0MU   0.925376            False   

                                         product_title  
73   Genesis GDP805P 5-Speed 2.6 Amp 8" Drill Press...  
70   Workbench Mounted Drilling Machine, 350W 5 Spe...  
76              Palmgren Ratcheting arbor press, 3 ton  
431  Creative Teaching Press Safari Friends Calenda...  
71   Drill Stand for Hand Drill, Electric Bench Cla...  
427  CTP Woodland Friends Calendar Set Bulletin Boa...  
77   Yost M7WW Rapid Acting Wood Working

## 3b: Dense model (all-MiniLM-L6-v2)


In [ ]:
# Setup, Constants, and Model Loading
MODEL_NAME = 'all-MiniLM-L6-v2'
DB_PATH = "./chroma_data"
COLLECTION_NAME = "product_embeddings"

model = SentenceTransformer(MODEL_NAME)
print("Model loaded")

Model loaded


In [ ]:
# Embed Documents for Indexing
documents = product_corpus_df['product_text'].tolist()
product_ids = product_corpus_df['product_id'].tolist()

print(f"Embedding {len(documents)}")
embeddings = model.encode(documents, show_progress_bar=True)
print("Embeddings generated.")

# Prepare data for ChromaDB
str_product_ids = [str(pid) for pid in product_ids]
metadatas = [{"product_id": pid, "text": doc} for pid, doc in zip(product_ids, documents)]

# Creating the ChromaDB Persistent Index
if os.path.exists(DB_PATH):
    shutil.rmtree(DB_PATH)
client = chromadb.PersistentClient(path=DB_PATH)

print(f"Creating ChromaDB collection '{COLLECTION_NAME}' at '{DB_PATH}'...")
collection = client.get_or_create_collection(
    name=COLLECTION_NAME, 
    metadata={"hnsw:space": "l2"} # 'l2' (Euclidean) distance
)

# adding the data
collection.add(
    embeddings=embeddings.tolist(),
    documents=documents,
    metadatas=metadatas,
    ids=str_product_ids # Chroma requires a list of unique string IDs
)
print(f"Successfully created collection with {collection.count()}")

Embedding 485


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Batches: 100%|██████████| 16/16 [00:06<00:00,  2.50it/s]


Embeddings generated.
Creating ChromaDB collection 'product_embeddings' at './chroma_data'...
Adding 485 vectors to collection...
Successfully created collection with 485 vectors.


In [ ]:
test_query = "carpenter bench press"

# ground truth for this query from our in-memory DataFrame
ground_truth_df = query_eval_set[query_eval_set['query'] == test_query]
ground_truth_ids = ground_truth_df['product_id'].tolist()
print(f"Test Query: '{test_query}'")
print(f"Ground Truth 'Exact' Product IDs ({len(ground_truth_ids)}): {ground_truth_ids}")

# Embed and search
query_vector = model.encode([test_query]).tolist()
n_neighbors = 10
print(f"\nSearching index for Top {n_neighbors} results...")
search_results = collection.query(
    query_embeddings=query_vector,
    n_results=n_neighbors,
)

print("Search Results (lower distance is better):")

# Chroma's output format
result_metadatas = search_results['metadatas'][0]
result_distances = search_results['distances'][0]

results_df = pd.DataFrame({
    'product_id': [meta['product_id'] for meta in result_metadatas],
    '_distance': result_distances
})

# Add a column to show if the result is a "Ground Truth" match
results_df['is_ground_truth'] = results_df['product_id'].isin(ground_truth_ids)


results_df = results_df.merge(product_corpus_df[['product_id', 'product_title']], on='product_id', how='left')
print(results_df[['product_id', '_distance', 'is_ground_truth', 'product_title']])

# metric for this query
matches_in_top_10 = results_df['is_ground_truth'].sum()
print(f"\nQuery-specific Metric: Found {matches_in_top_10} out of {len(ground_truth_ids)} ground truth items in the Top 10 results.")

print("\n--- Compare this to our TF-IDF Metric: Found 6 out of 8 ground truth items in the Top 10 results.")

Test Query: 'carpenter bench press'
Ground Truth 'Exact' Product IDs (8): ['B088R8VC7V', 'B07N4QN64D', 'B01M4F8JZJ', 'B01LZV1QW1', 'B00HQONFVE', 'B00BHSPJC8', 'B00068U7XQ', 'B00SIQ1DLS']

Searching index for Top 10 results...
Search Results (lower distance is better):
   product_id  _distance  is_ground_truth  \
0  B07N4QN64D   0.949831             True   
1  B088R8VC7V   1.055872             True   
2  B00HQONFVE   1.058271             True   
3  B00068U7XQ   1.136232             True   
4  B01LZV1QW1   1.194515             True   
5  B00SIQ1DLS   1.298030             True   
6  B07HH3K4SK   1.401191            False   
7  B01GOM6OUM   1.409457            False   
8  B00BHSPJC8   1.445602             True   
9  B078WZLFHG   1.452312            False   

                                       product_title  
0  Drill Stand for Hand Drill, Electric Bench Cla...  
1  Workbench Mounted Drilling Machine, 350W 5 Spe...  
2  WEN 4208T 2.3-Amp 8-Inch 5-Speed Benchtop Dril...  
3             P

## Eval

In [ ]:
def calculate_metrics(ground_truth_ids, retrieved_ids, k_values=[1, 5, 10]):
    """Calculate recall@k and MRR for a query"""
    metrics = {'hits': {}, 'mrr': 0.0}
    
    # hits@k for each k
    for k in k_values:
        top_k = retrieved_ids[:k]
        hits = len(set(ground_truth_ids) & set(top_k)) / len(ground_truth_ids)
        metrics['hits'][f'hits@{k}'] = hits
    
    # mean reciprocal rank
    mrr = 0.0
    for gt_id in ground_truth_ids:
        if gt_id in retrieved_ids:
            rank = retrieved_ids.index(gt_id) + 1
            mrr += 1.0 / rank
    
    metrics['mrr'] = mrr / len(ground_truth_ids) if ground_truth_ids else 0.0
    return metrics

def evaluate_tfidf_model(query, ground_truth_ids, tfidf_vectorizer, nn_index, product_corpus_df, k=10):
    """Run evaluation for TF-IDF model"""
    query_vector = tfidf_vectorizer.transform([query])
    distances, indices = nn_index.kneighbors(query_vector)
    result_indices = indices[0]
    
    retrieved_ids = product_corpus_df.iloc[result_indices]['product_id'].tolist()
    metrics = calculate_metrics(ground_truth_ids, retrieved_ids)
    metrics['query'] = query
    return metrics

def evaluate_dense_model(query, ground_truth_ids, model, collection, product_corpus_df, k=10):
    """Run evaluation for dense embedding model"""
    query_vector = model.encode([query]).tolist()
    
    search_results = collection.query(
        query_embeddings=query_vector,
        n_results=k,
    )
    
    retrieved_ids = [meta['product_id'] for meta in search_results['metadatas'][0]]
    metrics = calculate_metrics(ground_truth_ids, retrieved_ids)
    metrics['query'] = query
    return metrics

def print_avg_metrics(results, model_name):
    """Print average metrics across all queries"""
    avg_hits = {k: np.mean([r['hits'][k] for r in results]) for k in results[0]['hits'].keys()}
    avg_mrr = np.mean([r['mrr'] for r in results])
    
    print(f"\n{model_name} Average Metrics:")
    for k, v in avg_hits.items():
        print(f"{k.upper()}: {v:.3f}")
    print(f"MRR: {avg_mrr:.3f}")
    
    return avg_hits, avg_mrr

# build ground truth mapping
print("Creating ground truth mapping...")
ground_truth_map = {}
for _, row in query_eval_set.iterrows():
    query = row['query']
    product_id = row['product_id']
    
    if query not in ground_truth_map:
        ground_truth_map[query] = []
    ground_truth_map[query].append(product_id)

unique_queries = list(ground_truth_map.keys())
print(f"Found {len(unique_queries)} unique queries to evaluate.")

# run evaluations
print("\nRunning TF-IDF evaluation...")
tfidf_results = []
for query in tqdm(unique_queries, desc="TF-IDF Evaluation"):
    ground_truth_ids = ground_truth_map[query]
    metrics = evaluate_tfidf_model(query, ground_truth_ids, tfidf_vectorizer, nn_index, product_corpus_df)
    tfidf_results.append(metrics)

print("\nRunning Dense model evaluation...")
dense_results = []
for query in tqdm(unique_queries, desc="Dense Model Evaluation"):
    ground_truth_ids = ground_truth_map[query]
    metrics = evaluate_dense_model(query, ground_truth_ids, model, collection, product_corpus_df)
    dense_results.append(metrics)

print("\nEvaluation completed!")

# get average metrics
tfidf_hits, tfidf_mrr = print_avg_metrics(tfidf_results, "TF-IDF")
dense_hits, dense_mrr = print_avg_metrics(dense_results, "Dense Model")

# extract individual metrics for analysis
tfidf_mrrs = [r['mrr'] for r in tfidf_results]
tfidf_recalls_at_1 = [r['hits']['hits@1'] for r in tfidf_results]
tfidf_recalls_at_5 = [r['hits']['hits@5'] for r in tfidf_results]
tfidf_recalls_at_10 = [r['hits']['hits@10'] for r in tfidf_results]

dense_mrrs = [r['mrr'] for r in dense_results]
dense_recalls_at_1 = [r['hits']['hits@1'] for r in dense_results]
dense_recalls_at_5 = [r['hits']['hits@5'] for r in dense_results]
dense_recalls_at_10 = [r['hits']['hits@10'] for r in dense_results]

print("\nPer-Query Results:")
for i, query in enumerate(unique_queries):
    print(f"\nQuery: {query}")
    print("TF-IDF:", tfidf_results[i])
    print("Dense:", dense_results[i])

Creating ground truth mapping...
Found 50 unique queries to evaluate.

Running TF-IDF evaluation...


TF-IDF Evaluation: 100%|██████████| 50/50 [00:00<00:00, 786.68it/s]



Running Dense model evaluation...


Dense Model Evaluation: 100%|██████████| 50/50 [00:00<00:00, 80.00it/s]


Evaluation completed!

TF-IDF Average Metrics:
HITS@1: 0.130
HITS@5: 0.503
HITS@10: 0.689
MRR: 0.284

Dense Model Average Metrics:
HITS@1: 0.137
HITS@5: 0.554
HITS@10: 0.748
MRR: 0.308

Per-Query Results:

Query: 6 dining chairs
TF-IDF: {'hits': {'hits@1': 0.045454545454545456, 'hits@5': 0.22727272727272727, 'hits@10': 0.45454545454545453}, 'mrr': 0.13313492063492063, 'query': '6 dining chairs'}
Dense: {'hits': {'hits@1': 0.045454545454545456, 'hits@5': 0.22727272727272727, 'hits@10': 0.4090909090909091}, 'mrr': 0.12555916305916306, 'query': '6 dining chairs'}

Query: a intex pool pump
TF-IDF: {'hits': {'hits@1': 0.16666666666666666, 'hits@5': 0.8333333333333334, 'hits@10': 1.0}, 'mrr': 0.40833333333333327, 'query': 'a intex pool pump'}
Dense: {'hits': {'hits@1': 0.16666666666666666, 'hits@5': 0.8333333333333334, 'hits@10': 1.0}, 'mrr': 0.4083333333333334, 'query': 'a intex pool pump'}

Query: activated carbon mask
TF-IDF: {'hits': {'hits@1': 0.1, 'hits@5': 0.5, 'hits@10': 0.8}, 'mrr'

## Re-ranking



In [ ]:
print("Defining evaluation metrics...")

def calculate_reciprocal_rank(retrieved_ids, ground_truth_ids):
    """Get reciprocal rank for a query"""
    ground_truth_set = set(ground_truth_ids)
    for i, p_id in enumerate(retrieved_ids):
        if p_id in ground_truth_set:
            return 1.0 / (i + 1)
    return 0.0

def calculate_recall_at_k(retrieved_ids, ground_truth_ids, k):
    """Get recall@k for a query"""
    ground_truth_set = set(ground_truth_ids)
    retrieved_at_k = set(retrieved_ids[:k])
    
    hits = len(ground_truth_set.intersection(retrieved_at_k))
    return hits / len(ground_truth_set) if ground_truth_set else 0.0

print("Metric functions defined.")

# load models
print("\n--- Loading Models ---")

RE_RANKER_MODEL = 'cross-encoder/ms-marco-MiniLM-L-6-v2'
print(f"Loading re-ranking model: {RE_RANKER_MODEL}...")
cross_encoder_model = CrossEncoder(RE_RANKER_MODEL)
print("Re-ranker loaded.")

print(f"Loading retrieval model: {MODEL_NAME}...")
retrieval_model = SentenceTransformer(MODEL_NAME)
print("Retrieval model loaded.")

# run full evaluation (retrieve + re-rank)
print("\n--- Running Full Evaluation ---")

N_RETRIEVE = 50  # get more candidates for re-ranking
N_NEIGHBORS_FINAL = 10  # final top-k for evaluation

# build ground truth mapping
print("Creating ground truth map...")
ground_truth_map = query_eval_set.groupby('query')['product_id'].apply(list).to_dict()
unique_queries = list(ground_truth_map.keys())
print(f"Found {len(unique_queries)} unique queries to evaluate.")

# store results
rerank_mrrs = []
rerank_recalls_at_1 = []
rerank_recalls_at_5 = []
rerank_recalls_at_10 = []

# evaluate each query
for query in tqdm(unique_queries, desc="Evaluating (Retrieve + Re-rank)"):
    ground_truth_ids = ground_truth_map[query]
    
    # stage 1: retrieve candidates
    query_vector_dense = retrieval_model.encode([query]).tolist()
    
    search_results = collection.query( 
        query_embeddings=query_vector_dense,
        n_results=N_RETRIEVE,
    )
    
    # extract candidates
    retrieved_metadatas = search_results['metadatas'][0]
    candidate_ids = [meta['product_id'] for meta in retrieved_metadatas]
    candidate_texts = [meta['text'] for meta in retrieved_metadatas]

    # stage 2: re-rank with cross-encoder
    rerank_pairs = [(query, doc_text) for doc_text in candidate_texts]
    rerank_scores = cross_encoder_model.predict(rerank_pairs)
    
    # sort by new scores
    reranked_results = list(zip(candidate_ids, rerank_scores))
    reranked_results.sort(key=lambda x: x[1], reverse=True)
    retrieved_ids_reranked = [p_id for p_id, score in reranked_results]

    # calculate metrics on re-ranked results
    rerank_mrrs.append(calculate_reciprocal_rank(retrieved_ids_reranked, ground_truth_ids))
    rerank_recalls_at_1.append(calculate_recall_at_k(retrieved_ids_reranked, ground_truth_ids, k=1))
    rerank_recalls_at_5.append(calculate_recall_at_k(retrieved_ids_reranked, ground_truth_ids, k=5))
    rerank_recalls_at_10.append(calculate_recall_at_k(retrieved_ids_reranked, ground_truth_ids, k=N_NEIGHBORS_FINAL))

print("Re-ranking evaluation complete.")


--- Iteration 2: Re-ranking with a Cross-Encoder ---
Defining evaluation metrics...
Metric functions (MRR, HITS@k) defined.

--- Part 2: Loading Models and Clients ---
Loading re-ranking model: cross-encoder/ms-marco-MiniLM-L-6-v2...


c:\Users\saiasg\AppData\Local\miniconda3\envs\esci\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\saiasg\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L-6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed.

Re-ranker model loaded.
Loading retrieval model: all-MiniLM-L6-v2...
Retrieval model loaded.

--- Part 3: Running Full Evaluation (Retrieve + Re-rank) ---
Creating ground truth map...
Found 50 unique queries to evaluate.


Evaluating (Retrieve + Re-rank): 100%|██████████| 50/50 [01:50<00:00,  2.20s/it]

Re-ranking complete.


In [22]:
print("\n--- TF-IDF (Baseline) ---")
print(f"  Mean Reciprocal Rank (MRR): {np.mean(tfidf_mrrs):.4f}") 
print(f"  HITS@10 (Recall@10):        {np.mean(tfidf_recalls_at_10):.4f}")

print("\n--- Dense Model (Baseline) ---")
print(f"  Mean Reciprocal Rank (MRR): {np.mean(dense_mrrs):.4f}") 
print(f"  HITS@10 (Recall@10):        {np.mean(dense_recalls_at_10):.4f}")

print("\n--- Re-ranked Dense Model (Iteration 2) ---")
print(f"  Mean Reciprocal Rank (MRR): {np.mean(rerank_mrrs):.4f}")
print(f"  HITS@1 (Recall@1):          {np.mean(rerank_recalls_at_1):.4f}")
print(f"  HITS@5 (Recall@5):          {np.mean(rerank_recalls_at_5):.4f}")
print(f"  HITS@10 (Recall@10):        {np.mean(rerank_recalls_at_10):.4f}")


--- TF-IDF (Baseline) ---
  Mean Reciprocal Rank (MRR): 0.2843
  HITS@10 (Recall@10):        0.6891

--- Dense Model (Baseline) ---
  Mean Reciprocal Rank (MRR): 0.3076
  HITS@10 (Recall@10):        0.7483

--- Re-ranked Dense Model (Iteration 2) ---
  Mean Reciprocal Rank (MRR): 0.9519
  HITS@1 (Recall@1):          0.1381
  HITS@5 (Recall@5):          0.5641
  HITS@10 (Recall@10):        0.7858


# Step 4 Two tower eval vs others

Refrence links:

Uber blog on two tower arch: https://www.uber.com/blog/innovative-recommendation-applications-using-two-tower-embeddings/

In [30]:
print("=== Two-Tower Model Evaluation ===")

# Import required libraries
import sys
import os
from two_tower_evaluation import TwoTowerEvaluator

# Check if model exists
model_path = './two_tower_model'
if not os.path.exists(model_path):
    print("❌ Two-Tower model not found. Please train the model first using two_tower_fixed.py")
else:
    print("✅ Two-Tower model found. Starting evaluation...")
    
    try:
        # Initialize evaluator
        evaluator = TwoTowerEvaluator(
            model_path=model_path,
            test_data_path='sample_2c_full_data.csv'
        )
        
        # Run evaluation
        print("\n🔄 Building product embeddings index...")
        evaluator.build_product_index()
        
        # Test the same query as baselines for comparison
        test_query = "carpenter bench press"
        print(f"\n🔍 Testing query: '{test_query}'")
        
        # Get ground truth for comparison
        ground_truth_df = evaluator.eval_data[evaluator.eval_data['query'] == test_query]
        ground_truth_ids = set(ground_truth_df[ground_truth_df['esci_label'] == 'E']['product_id'].tolist())
        
        print(f"📋 Ground Truth: {len(ground_truth_ids)} relevant products")
        print(f"    Product IDs: {list(ground_truth_ids)}")
        
        # Search using Two-Tower model
        results = evaluator.search_products(test_query, top_k=10)
        
        print(f"\n🎯 Two-Tower Top-10 Results:")
        hits = 0
        for i, (product_id, similarity) in enumerate(results, 1):
            is_relevant = product_id in ground_truth_ids
            if is_relevant:
                hits += 1
            
            # Get product title
            product_title = evaluator.product_corpus[evaluator.product_corpus['product_id'] == product_id]['product_title'].iloc[0]
            status = "✅ RELEVANT" if is_relevant else "❌"
            
            print(f"  {i:2d}. {status} | Sim: {similarity:.4f} | {product_title[:60]}...")
        
        two_tower_precision = hits / 10
        two_tower_recall = hits / len(ground_truth_ids) if ground_truth_ids else 0
        
        print(f"\n📊 Two-Tower Results:")
        print(f"    Precision@10: {two_tower_precision:.4f}")
        print(f"    Recall@10: {two_tower_recall:.4f}")
        print(f"    Found {hits} out of {len(ground_truth_ids)} ground truth items in Top 10")
        
        # Compare with previous results
        print(f"\n📈 Comparison Summary:")
        print(f"    TF-IDF:              Found 6/8 ground truth items (Precision: 0.6000)")
        print(f"    SentenceTransformers: Found 7/8 ground truth items (Precision: 0.7000)")
        print(f"    Two-Tower:           Found {hits}/{len(ground_truth_ids)} ground truth items (Precision: {two_tower_precision:.4f})")
        
        # Run comprehensive evaluation
        print(f"\n🔄 Running comprehensive evaluation on all test queries...")
        metrics = evaluator.calculate_metrics([1, 5, 10, 20])
        avg_metrics, _ = metrics
        
        print(f"\n📈 Comprehensive Results:")
        print(f"{'Metric':<12} {'Top-1':<8} {'Top-5':<8} {'Top-10':<8} {'Top-20':<8}")
        print("-" * 48)
        
        for metric_name in ['precision', 'recall', 'ndcg', 'mrr']:
            row = f"{metric_name.upper():<12}"
            for k in [1, 5, 10, 20]:
                row += f"{avg_metrics[k][metric_name]:<8.3f}"
            print(row)
        
        # Sample additional queries
        print(f"\n🔍 Testing additional sample queries...")
        sample_queries = ['6 dining chairs', 'plants', 'turning shoe']
        
        for query in sample_queries:
            if query in evaluator.eval_data['query'].values:
                ground_truth = evaluator.eval_data[evaluator.eval_data['query'] == query]
                relevant_count = len(ground_truth[ground_truth['esci_label'] == 'E'])
                
                results = evaluator.search_products(query, top_k=5)
                relevant_in_top5 = sum(1 for pid, _ in results if pid in set(ground_truth[ground_truth['esci_label'] == 'E']['product_id']))
                
                print(f"  '{query}': {relevant_in_top5}/{relevant_count} relevant in Top-5")
        
        print(f"\n✅ Two-Tower evaluation completed!")
        
    except Exception as e:
        print(f"❌ Error during evaluation: {e}")
        import traceback
        traceback.print_exc()

print("\n" + "="*60)
print("🏁 FINAL COMPARISON SUMMARY")
print("="*60)
print("Method                | Precision@10 | Recall@10 | Notes")
print("-" * 60)
print("TF-IDF               |    0.6000    |   0.7500  | Sparse, keyword-based")
print("SentenceTransformers |    0.7000    |   0.8750  | Dense, pre-trained")
print("Two-Tower (Custom)   |    ?.????    |   ?.????  | Dense, task-specific")
print("\n💡 The Two-Tower model should perform better due to:")
print("   - Task-specific training on your exact data")
print("   - Separate encoders for queries and products")
print("   - Triplet loss optimization for retrieval")


=== Two-Tower Model Evaluation ===
✅ Two-Tower model found. Starting evaluation...
Preparing evaluation data...
Product corpus: 485 unique products
Evaluation data: 485 query-product pairs
Unique queries: 50 queries

🔄 Building product embeddings index...
Building product embeddings index...


Encoding products: 100%|██████████| 16/16 [00:29<00:00,  1.85s/it]


Product index built: (485, 256)

🔍 Testing query: 'carpenter bench press'
📋 Ground Truth: 8 relevant products
    Product IDs: ['B07N4QN64D', 'B088R8VC7V', 'B01M4F8JZJ', 'B00068U7XQ', 'B01LZV1QW1', 'B00HQONFVE', 'B00SIQ1DLS', 'B00BHSPJC8']


Encoding querys: 100%|██████████| 1/1 [00:00<00:00,  8.69it/s]



🎯 Two-Tower Top-10 Results:
   1. ❌ | Sim: 0.3707 | Toddler Kids Baby Girl Ruffle 3/4 Long Flare Sleeve Floral P...
   2. ❌ | Sim: 0.3591 | White and Orange and Medium Yellow Latex Balloons...
   3. ❌ | Sim: 0.3539 | 80 Pieces Mardi Gras Balloons Latex Balloons Confetti Balloo...
   4. ❌ | Sim: 0.3389 | Weileenice 3-14Y Flower Girl Wedding Dresses 3T 4T Toddler C...
   5. ❌ | Sim: 0.3297 | Todaies Women Cardigan Coat, Women Long Sleeve Loose Coat Ov...
   6. ❌ | Sim: 0.3254 | AYHome Kids Animal Onesie Unicorn Costume Christmas Hallowee...
   7. ❌ | Sim: 0.3254 | Fancy Ivory White Lace Flower Girl Dress Boho Rustic First C...
   8. ❌ | Sim: 0.3251 | Whaline 80Pcs Fall Balloon Set Autumn Leave Maple Leave Pump...
   9. ❌ | Sim: 0.3214 | Simplee Women's Casual Open Front Long Sleeve Knit Cardigan ...
  10. ❌ | Sim: 0.3188 | Tueenhuge Baby Girls Christmas Dress Toddler Snowflake Print...

📊 Two-Tower Results:
    Precision@10: 0.0000
    Recall@10: 0.0000
    Found 0 out of 8 ground truth

...

  '6 dining chairs': 0/22 relevant in Top-5


Encoding querys: 100%|██████████| 1/1 [00:00<00:00, 17.34it/s]


  'plants': 0/22 relevant in Top-5


Encoding querys: 100%|██████████| 1/1 [00:00<00:00, 15.87it/s]

  'turning shoe': 0/30 relevant in Top-5

✅ Two-Tower evaluation completed!

🏁 FINAL COMPARISON SUMMARY
Method                | Precision@10 | Recall@10 | Notes
------------------------------------------------------------
TF-IDF               |    0.6000    |   0.7500  | Sparse, keyword-based
SentenceTransformers |    0.7000    |   0.8750  | Dense, pre-trained
Two-Tower (Custom)   |    ?.????    |   ?.????  | Dense, task-specific

💡 The Two-Tower model should perform better due to:
   - Task-specific training on your exact data
   - Separate encoders for queries and products
   - Triplet loss optimization for retrieval
